In [3]:
# coding: utf-8

# In[ ]:


import requests             
from bs4 import BeautifulSoup 
import csv                  
import webbrowser
import io

def display(content, filename='output.html'):
    with open(filename, 'wb') as f:
        f.write(content)
        webbrowser.open(filename)

def get_soup(session, url, show=False):
    r = session.get(url)
    if show:
        display(r.content, 'temp.html')

    if r.status_code != 200: # not OK
        print('[get_soup] status code:', r.status_code)
    else:
        return BeautifulSoup(r.text, 'html.parser')
    
def post_soup(session, url, params, show=False):
    '''Read HTML from server and convert to Soup'''

    r = session.post(url, data=params)
    
    if show:
        display(r.content, 'temp.html')

    if r.status_code != 200: # not OK
        print('[post_soup] status code:', r.status_code)
    else:
        return BeautifulSoup(r.text, 'html.parser')
    
def scrape(url, lang='ALL'):

    # create session to keep all cookies (etc.) between requests
    session = requests.Session()

    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:57.0) Gecko/20100101 Firefox/57.0',
    })


    items = parse(session, url + '?filterLang=' + lang)

    return items

def parse(session, url):
    '''Get number of reviews and start getting subpages with reviews'''

    print('[parse] url:', url)

    soup = get_soup(session, url)

    if not soup:
        print('[parse] no soup:', url)
        return

    #num_reviews = soup.find('span', class_='reviews_header_count').text # get text
    num_reviews = soup.find('span', class_='reviews_header_count')
    num_reviews = num_reviews.text
    num_reviews = num_reviews[1:-1] 
    num_reviews = num_reviews.replace(',', '')
    num_reviews = int(num_reviews) # convert text into integer
    print('[parse] num_reviews ALL:', num_reviews)

    url_template = url.replace('.html', '-or{}.html')
    print('[parse] url_template:', url_template)

    items = []

    offset = 0

    while(True):
        subpage_url = url_template.format(offset)

        subpage_items = parse_reviews(session, subpage_url)
        if not subpage_items:
            break

        items += subpage_items

        if len(subpage_items) < 5:
            break

        offset += 5

    return items

def get_reviews_ids(soup):

    items = soup.find_all('div', attrs={'data-reviewid': True})

    if items:
        reviews_ids = [x.attrs['data-reviewid'] for x in items][::2]
        print('[get_reviews_ids] data-reviewid:', reviews_ids)
        return reviews_ids
    
def get_more(session, reviews_ids):

    url = 'https://www.tripadvisor.com/OverlayWidgetAjax?Mode=EXPANDED_HOTEL_REVIEWS_RESP&metaReferer=Hotel_Review'

    payload = {
        'reviews': ','.join(reviews_ids), # ie. "577882734,577547902,577300887",
        #'contextChoice': 'DETAIL_HR', # ???
        'widgetChoice': 'EXPANDED_HOTEL_REVIEW_HSX', # ???
        'haveJses': 'earlyRequireDefine,amdearly,global_error,long_lived_global,apg-Hotel_Review,apg-Hotel_Review-in,bootstrap,desktop-rooms-guests-dust-en_US,responsive-calendar-templates-dust-en_US,taevents',
        'haveCsses': 'apg-Hotel_Review-in',
        'Action': 'install',
    }

    soup = post_soup(session, url, payload)

    return soup

def parse_reviews(session, url):
    '''Get all reviews from one page'''

    print('[parse_reviews] url:', url)

    soup =  get_soup(session, url)

    if not soup:
        print('[parse_reviews] no soup:', url)
        return

    hotel_name = soup.find('h1', id='HEADING').text

    reviews_ids = get_reviews_ids(soup)
    if not reviews_ids:
        return

    soup = get_more(session, reviews_ids)

    if not soup:
        print('[parse_reviews] no soup:', url)
        return

    items = []

    for idx, review in enumerate(soup.find_all('div', class_='reviewSelector')):

        badgets = review.find_all('span', class_='badgetext')
        if len(badgets) > 0:
            contributions = badgets[0].text
        else:
            contributions = '0'

        if len(badgets) > 1:
            helpful_vote = badgets[1].text
        else:
            helpful_vote = '0'
        user_loc = review.select_one('div.userLoc strong')
        if user_loc:
            user_loc = user_loc.text
        else:
            user_loc = ''
            
        bubble_rating = review.select_one('span.ui_bubble_rating')['class']
        bubble_rating = bubble_rating[1].split('_')[-1]
        #print (bubble_rating)
        item = {
            'review_body': review.find('p', class_='partial_entry').text,
            #'review_date': review.find('span', class_='ratingDate')['title'], # 'ratingDate' instead of 'relativeDate'
            'review_rating' : bubble_rating
        }

        items.append(item)
        print('\n--- review ---\n')
        for key,val in item.items():
            print(' ', key, ':', val)

    print()

    return items

def write_in_csv(items, filename='results.csv',
                  headers=['hotel name', 'review title', 'review body',
                           'review date', 'contributions', 'helpful vote',
                           'user name' , 'user location', 'rating'],
                  mode='w'):

    print('--- CSV ---')

    with io.open(filename, mode, encoding="utf-8") as csvfile:
        csv_file = csv.DictWriter(csvfile, headers)

        if mode == 'w':
            csv_file.writeheader()

        csv_file.writerows(items)
        
DB_COLUMN   = 'review_body'
DB_COLUMN1 = 'review_date'
DB_COLUMN2 = 'review_rating'

start_urls = [
#     'https://www.tripadvisor.com/Attraction_Review-g297701-d1515658-Reviews-Tegalalang_Rice_Terrace-Ubud_Gianyar_Bali.html',
#     'https://www.tripadvisor.com/Attraction_Review-g297699-d6394471-Reviews-Yeh_Sanih_Spring-Singaraja_Bali.html',
#     'https://www.tripadvisor.com/Attraction_Review-g2283829-d379340-Reviews-Tanah_Lot_Temple-Beraban_Bali.html',
#     'https://www.tripadvisor.com/Attraction_Review-g294226-d4226508-Reviews-Batur_Natural_Hotspring-Bali.html',
#     'https://www.tripadvisor.com/Attraction_Review-g1219108-d1436488-Reviews-Garuda_Wisnu_Kencana_Cultural_Park-Ungasan_Nusa_Dua_Peninsula_Bali.html',
    'https://www.tripadvisor.com/Attraction_Review-g608501-d1379504-Reviews-Ampera_Bridge-Palembang_South_Sumatra_Sumatra.html',
]

lang = 'id'

headers = [ 
    DB_COLUMN, 
    #DB_COLUMN1, 
    DB_COLUMN2,
]

for url in start_urls:

    # get all reviews for 'url' and 'lang'
    items = scrape(url, lang)

    if not items:
        print('No reviews')
    else:
        # write in CSV
        print("else")
        filename = url.split('Reviews-')[1][:-5] + '__' + lang
        print('filename:', filename)
        write_in_csv(items, filename + '.csv', headers, mode='w')
        
#span class="ui_bubble_rating bubble_30" buat cari rating


[parse] url: https://www.tripadvisor.com/Attraction_Review-g608501-d1379504-Reviews-Ampera_Bridge-Palembang_South_Sumatra_Sumatra.html?filterLang=id
[parse] num_reviews ALL: 793
[parse] url_template: https://www.tripadvisor.com/Attraction_Review-g608501-d1379504-Reviews-Ampera_Bridge-Palembang_South_Sumatra_Sumatra-or{}.html?filterLang=id
[parse_reviews] url: https://www.tripadvisor.com/Attraction_Review-g608501-d1379504-Reviews-Ampera_Bridge-Palembang_South_Sumatra_Sumatra-or0.html?filterLang=id
[get_reviews_ids] data-reviewid: ['686311608', '682456464', '680313634', '677009686', '665970867', '664900023', '664116501', '664107970', '658403469', '653591364']

--- review ---

  review_body : Menikmati jembatan ampera sambil mengabadikan sunrise dan sunset bersama teman" menjadi bigitu menyenangkan
  review_rating : 50

--- review ---

  review_body : Jembatan ikonik yang sangat identik dengan kota Palembang. Titik teramai sekaligus terbaik untuk menikmati pemandangan jembatan ini adalah 

[get_reviews_ids] data-reviewid: ['653109393', '649877932', '649289918', '647553920', '647263548', '646752402', '641694267', '641177051', '639999759', '639119608']

--- review ---

  review_body : Jembatan Ampera merupakan ICON dari kota Palembang, di malam hari Jembatan ini sangat indah karena di Hiasi oleh Lampu yang berwarna-warni. dari arah Benteng Kuto Besak kita dapat menikmati Pesona Indahnya Jembatan Ampera sekaligus semilir angin dari Pinggir Sungai Musi.
  review_rating : 50

--- review ---

  review_body : Jembatan yang terletak diatas sungai musi, jembatan yang merupakan salah satu icon kota palembang, saat malam jembatan akan terlihat sangat indah, agak dekat dengan pasar 16, bkb, masjid agung.
  review_rating : 40

--- review ---

  review_body : Salah satu objek wisata terkenal dikota palembang, bisa di bilang ya iconnya kota palembang, selain empek empek ya, lokasi mudah diakses kendaraan umum, aman cuma terkandala lokasi agak macet, dekat dengan pasar legenda di palemb

[get_reviews_ids] data-reviewid: ['638957309', '638477905', '637578715', '637529026', '633674512', '632528726', '631868236', '631770122', '631533535', '630481530']

--- review ---

  review_body : Jembatan ampera keren bgt apalagi klo mlm hari bt bfoto krn lampu2nya,,qt k dermaga bkb ad perahu apung yg jualan pempek dsana enak dan termasuk murah.ga rugi wisata malam ke dermaga bkb dekat dengan jembatan amper
  review_rating : 50

--- review ---

  review_body : Ini sudah ke sekian kali saya ke Palembang, rasanya belum lengkap dan terasa mengganjal bila belum ke Jembatan Ampera di malam hari. Apapun urusannya di Palembang, jembatan Ampera adalah tempat yang patut dikunjungi, bagai saudarayang jauh, tidak elok bila tidak bertandang ke saudara tersebut.Saya melihat sudah banyak perkembangan destinasi ini, yang membuat pengunjung betah dan bermalam-malam disana. Apalagi kalau pas hari Sabtu Malam atau Munggu Malam. Tempat yang cocok untuk pertemanan dan bernostalgia memadu rindu berbagai p


--- review ---

  review_body : Alhamdulillah...akhirnya sampai juga deh di Palembang..Ada yang kalian rasakan tidak ya pada saat mendaratkan kaki turun dari pesawat di Bandara Sultan Mahmud Badaruddin II, Ada ga ? hayoooo Jujur....Serasa ada yang bisik-bisik " Ayooo..jangan lupa foto ya nanti saat tiba di Palembang dengan latar belakang Jembatan Ampera, terserah kamu mau fotonya pagi, siang, sore atau malam pasti ga ada kecewanya " Nah, mulai merasa khan? heheh..Tapi memang Jembatan yang satu ini ga bisa ditinggalkan dari sejarahnya Palembang (ciyeeee..sok tahu banget ya saya, hahaha).Yang pasti rasa ingin berlama-lama di Jembatan ini ga mau berakhir, gimana engga? di bawahnya sungai Musi yang bersih dan banyak juga perahu-perahu yang berlalu lalang, dan ditambah lagi area untuk pejalan kakinya udah bagus banget, dibangun dengan manja, ada aksen untuk orang-orang duduk dan bersantai sepanjang jembatan.Owh iya, jembatannya itu sekarang ada JAM DINDING besarnya, makin KECEHHH Dehh IKON

[get_reviews_ids] data-reviewid: ['589633646', '589338983', '588767351', '588590088', '586762631', '586088481', '576158624', '575780264', '573906570', '572763621']

--- review ---

  review_body : Tempat ini adalah icon palembang, asik untuk take a picture. Tapi perhatikan barang berharga anda, jangan terlalu berlebihan ditempat ini
  review_rating : 50

--- review ---

  review_body : Salah satu ikon kota Palembang ada beberapa sudut yang sangat cantik untuk photo di jembatan ampera seperti berada di dermaga kfc atau di klenteng 10 ulu bahkan bisa juga dari river side atau kampung kapitan
  review_rating : 50

--- review ---

  review_body : Icon kota palembang,,kebanggaan wong kito,jembatan sungai musi atau biasa di sebut jembatan AMPERA membentang di tengah kota palembang,penghubung ilir dan ulu kota palembang,,sekarang tampilan nya lebih berwarna warni di hias cahaya lampu merah
  review_rating : 50

--- review ---

  review_body : Jembatan bersejarah di Palembang. Sejak kecil ingi

...

[get_reviews_ids] data-reviewid: ['203396610', '203319550', '203308673', '203246742', '202502070', '202358024', '200648946', '200537311', '200473848', '200361726']

--- review ---

  review_body : cuaca agak gerimis saat saya mulai mengambil gambar jembatan Ampera ini, dan sambil ngopi ngopi disamping sungai Musi,, membawa pengalaman tersendiri
  review_rating : 40

--- review ---

  review_body : kemajuan pesat, dibanding terdahulu dan cukup keren. lbh aman dan bersih. Di saat acara tertentu,banyak atraksi barongsay dan aneka makanan,minuman dan souvenir
  review_rating : 30

--- review ---

  review_body : Jembatan terpanjang dan terkokoh di sumatera, dibangun dijaman penjajahan jepang dan sampai sekarang masih kokoh untuk menjembatani sungai musi
  review_rating : 40

--- review ---

  review_body : Senang bisa menyusuri sungai musi dengan perahu yang terombang-ambing karena gelombang airnya cukup besar, sampai-sampai baju pada basah. Tapi sayang air sungai musi itu kotor dan banyak

[get_reviews_ids] data-reviewid: ['200215761', '200043370', '199101147', '198584727', '197920061', '196975947', '196901734', '196696913', '190886981', '187331068']

--- review ---

  review_body : Layaknya kota lain, kota Palembang juga mempunyai landmark yakni Jembatan Ampera. Melewati pusat kota ini dari atas jembatan terlihat banyak masyarakat menghabiskan sore di pinggir sungai Musi. Tepatnya di depan sebuah benteng peninggalan Belanda banyak tempat-tempat asyik untuk nongkrong. Kawasan ini menjadi pusat masyarakat Palembang untuk bersosialisasi. Sambil menikmati jajanan ringan kita bisa menikmati hiruk pikuk disekitar jembatan yang membentang tinggi membelah sungai Musi.
  review_rating : 30

--- review ---

  review_body : Jembatan yang sudah tidak terlihat menarik lagi karena kurangnya perhatian dari pemerintah setempat, sehingga dibawah jembatan banyak terlihat sampah dan kerumunan pedagang yang tidak memikirkan efisiensi kenyamanan suatu tempat.
  review_rating : 30

--- revie

[get_reviews_ids] data-reviewid: ['187241615', '186888111', '186609580', '176180209', '175741964', '175375923', '173295935', '167506179', '165717400', '164234956']

--- review ---

  review_body : Cantiknya ampera di malam hari.. Saran saya kunjjungi ampera di malam hari.. Pesona ampera dengan lampu hiasanya meenjadikan ampera sangat mempesona.
  review_rating : 50

--- review ---

  review_body : Jembatan ini terletak di tengah-tengah kota Palembang, menghubungkan daerah Seberang Ulu dan Seberang Ilir yang dipisahkan oleh Sungai MusiDisarankan untuk datang ke sini pada malam hari karena cahaya2 lampu di malam hari yg menghiasi jembatan dan sekitarnya sungguh sangat indah
  review_rating : 40

--- review ---

  review_body : jembatan ampera merupakan salah satu maskot kota Palembang, dijembatan ini banyak wisatawan berfoto, jembatan ampera penghubung antara daerah hilir dan hulu dari kota palembang yg dibangun pada masa pemerintahan jepang.jembatan ini akan terlihat indah pada malam ha

[get_reviews_ids] data-reviewid: ['162244223', '160222769', '158197159', '155457896', '152148425', '151707507', '150225098', '148185717', '146979821', '146945948']

--- review ---

  review_body : Saya kagum melihat Jembatan Ampera di malam hari, terlihat indah dihiasi lampu warna warni, icon palembang ini salah satu tempat tujuan wisata malam palembang. recomended :)
  review_rating : 40

--- review ---

  review_body : Peninggalan sejarah yang harus terus di jaga.Selain itu jembatan ini juga memberikan fungsi transportasi yang berarti bagi warga Kota Palembang..
  review_rating : 30

--- review ---

  review_body : Ampera yang di buat oleh negara jepang karena kota palembang (indonesia) telah memenangkan perang yang terjadi pada tahun 1945 selama 5 hari 5 malam terjadinya perebutan wilayah di daerah kami dengan negra lain yaitu jepang
  review_rating : 50

--- review ---

  review_body : kapankan jembatan ampera dapat berfungsi kembali? agar dapat dilalui oleh kapal kapal besar menuj

[get_reviews_ids] data-reviewid: ['144621250', '144204363', '142615523', '139574366', '139566610', '138168957', '137999161', '134572439', '129339282', '127960716']

--- review ---

  review_body : jembatannya bagus dan menarik...cuman itu membuat hatiku sedih kok kotor yah??kotornya itu ada banyak sampah sampah itu berserakan dimana mana...tempatnya selalu rame..disekitar jembatan ini ada banyak angkutan angkutan berkeliaran.. jadi kelihatan kurang rapi~
  review_rating : 50

--- review ---

  review_body : menyaksikan pemandangan kota palembang diwaktu malam sangatlah indah,apalagi dari atas jembatan Ampera.Kita Bisa lihat gemerlap lampu warna warni. Tampak dari kejauhan sorot lampu laser dari berbagai sudut di kota palembang,menambah keindahan pemandangan palembang di waktu malam.
  review_rating : 40

--- review ---

  review_body : duduk makan diLesehan, melihat jebatan dari jarak pandang yang cukup dekat dimalam hari ditemani sang kekasih dengan ditemani secangkir kopi hangat sert